# Student Loan Risk with Deep Learning

In [17]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
import numpy as np

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [18]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [19]:
# Check the credit_ranking value counts
display (loans_df["credit_ranking"].value_counts())
# another way of looking at the feature. not prefered as this does not show the count per class
display (loans_df['credit_ranking'].unique())

credit_ranking
1    855
0    744
Name: count, dtype: int64

array([0, 1])

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [20]:
# Define the target set y using the credit_ranking column
y =loans_df["credit_ranking"]

# Display a sample of y
np.array(y)

array([0, 0, 0, ..., 1, 0, 1])

In [21]:
# Define features set X by selecting all columns but credit_ranking
X=loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
display (X.head())


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [22]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [24]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Transform the training and test datasets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Display the first few rows of the scaled training data
print(pd.DataFrame(X_train_scaled, columns=X_train.columns).head())


   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0        -0.733079            0.664893          -1.257044    -0.320459   
1         1.067741           -0.623462           1.523148     0.608863   
2        -1.746040           -1.071585          -1.358142    -0.534917   
3        -0.620528            0.496847          -1.054849    -0.034513   
4        -0.507977            0.608877          -1.004300    -0.534917   

   alumni_success  study_major_code  time_to_completion  \
0       -0.453622         -0.742407           -0.645507   
1       -0.369546         -1.125190           -1.112003   
2       -0.789922          1.075808            0.536281   
3       -0.201396          0.118853            1.189375   
4        0.261018         -0.742407           -0.707707   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                0.240001        0.988460          0.063095   
1                0.187899       -1.753513         -0.173904   
2          

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [25]:
# Define the the number of inputs (features) to the model

num_features = X_train.shape[1]

# Review the number of features
X_train.shape[1]

11

In [29]:
11
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 16

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 8


# Define the number of neurons in the output layer
output_neurons = 1


In [28]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer

model.add(Dense(units=hidden_nodes_layer1, activation='relu', input_dim=num_features))

# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))


# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation='sigmoid'))


In [30]:
# Display the Sequential model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                192       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 337 (1.32 KB)
Trainable params: 337 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [31]:
# Compile the Sequential model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [32]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
32/32 [==============================] - 2s 11ms/step - loss: 0.7384 - accuracy: 0.5112 - val_loss: 0.6950 - val_accuracy: 0.5352
Epoch 2/50
32/32 [==============================] - 0s 6ms/step - loss: 0.6850 - accuracy: 0.6051 - val_loss: 0.6544 - val_accuracy: 0.6172
Epoch 3/50
32/32 [==============================] - 0s 6ms/step - loss: 0.6510 - accuracy: 0.6569 - val_loss: 0.6240 - val_accuracy: 0.7031
Epoch 4/50
32/32 [==============================] - 0s 4ms/step - loss: 0.6221 - accuracy: 0.6745 - val_loss: 0.5979 - val_accuracy: 0.7305
Epoch 5/50
32/32 [==============================] - 0s 4ms/step - loss: 0.5977 - accuracy: 0.6901 - val_loss: 0.5792 - val_accuracy: 0.7188
Epoch 6/50
32/32 [==============================] - 0s 5ms/step - loss: 0.5786 - accuracy: 0.7058 - val_loss: 0.5646 - val_accuracy: 0.7188
Epoch 7/50
32/32 [==============================] - 0s 4ms/step - loss: 0.5629 - accuracy: 0.7116 - val_loss: 0.5530 - val_accuracy: 0.7383
Epoch 8/50
32/32 [=

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [33]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data

loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {loss}, Accuracy: {accuracy}")


10/10 - 0s - loss: 0.5135 - accuracy: 0.7437 - 59ms/epoch - 6ms/step
Loss: 0.5135123133659363, Accuracy: 0.7437499761581421


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [34]:
# Set the model's file path
file_path = Path("student_loans.keras")


# Export your model to a keras file
model.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [35]:
# Set the model's file path

file_path = Path("student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [36]:
# Make predictions with the test data
predictions = model.predict(X_test_scaled)


# Display a sample of the predictions
sample_predictions = predictions[:5]
print(sample_predictions)


10/10 [==============================] - 0s 7ms/step
[[0.5714339 ]
 [0.39209488]
 [0.89159936]
 [0.7857904 ]
 [0.98473024]]


In [37]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=['Prediction'], data=predictions)
predictions_df["Prediction"] = round(predictions_df["Prediction"],0)
predictions_df

,Prediction
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
315,1.0
316,0.0
317,0.0
318,1.0


### Step 4: Display a classification report with the y test data and predictions

In [38]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["Prediction"].values))


              precision    recall  f1-score   support

           0       0.73      0.73      0.73       154
           1       0.75      0.75      0.75       166

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

Gathering student demographics (age, gender, location, family income, parental education levels) provides socio-economic context, while academic information (GPA, major, school ranking, expected graduation date) assesses academic standing and future income potential. Loan history (previous loans, repayment history, loan amounts) offers insight into borrowing behavior and creditworthiness. Financial aid information (scholarships, grants, work-study programs) helps evaluate current financial support, and employment details (part-time jobs, internships, job offers) predict loan repayment ability. Additionally, understanding preferences for loan types, repayment terms, and deferment options ensures tailored recommendations. This comprehensive data collection is crucial for recommending suitable loan options based on the student's financial situation, academic prospects, and borrowing behavior.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

The data includes detailed student attributes like academic performance, financial status, and personal preferences, enabling personalized loan recommendations based on individual profiles and needs. Content-based filtering is ideal for recommending loans tailored to each student's unique circumstances, even without similar user data. While collaborative filtering could complement this approach if enough data on student interactions with loans is available, content-based filtering is more suitable given the detailed and varied nature of the attributes.


3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

Concern 1:
Handling sensitive personal information, such as financial data, academic records, and personal demographics, requires robust measures for data privacy and security, including strong encryption, access controls, and compliance with regulations like GDPR and FERPA. Mishandling data can lead to legal consequences and reputational damage, making compliance essential to avoid penalties and maintain trust. Secure data handling fosters trust and adoption among students and families, while breaches or misuse can erode trust and reduce system usage, potentially harming the institution's reputation.

Concern 2: Student financial needs can change rapidly due to factors like changes in family income, unexpected expenses, shifts in academic plans, or new scholarships. The recommendation system must be adaptable to these changes to provide accurate and timely loan recommendations. If the system cannot promptly update and adapt, its recommendations may become outdated or irrelevant, potentially leading to financial strain or missed opportunities for students. Additionally, a system that fails to account for dynamic changes may frustrate users, reducing their engagement and satisfaction. Students expect recommendations that reflect their current circumstances, so the system must offer real-time updates and continuous learning. Addressing these challenges ensures data integrity and relevance, providing more effective and trustworthy loan recommendations.